Create json file of UK recessionary periods
1. open `GDP` sheet of `datadownload.xlsx`
2. create column with 1 if UK in recession, 0 if not
    - use `quarterly change (%)` column, recession is defined as two consecutive quarters of negative growth
3. find start & end timestamps of each recession

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('../altair_wrapper/data/datadownload.xlsx', sheet_name='GDP')

# Add column if UK is in recession, defined as two consecutive quarters of negative growth
df['recession'] = ((df['quarterly change (%)'] < 0) & (df['quarterly change (%)'].shift(1) < 0)) | ((df['quarterly change (%)'] < 0) & (df['quarterly change (%)'].shift(-1) < 0))

In [ ]:
# Convert `date` column from 'yyyy qq' format to datetime (yyyy-mm-dd)
quarters = {
    'Q1': '-01-01',
    'Q2': '-04-01',
    'Q3': '-07-01',
    'Q4': '-10-01'
}

df.columns = ['quarter', 'growth', 'recession']
# replace `date` column according to quarters dict

df['date'] = df['quarter'].str.replace(' ', '').replace(quarters, regex=True)

In [ ]:
df.to_csv('recessions_uk.csv', index=False)

In [ ]:
# Function to find start and end of recessions
def find_recession_periods(df):
    recession_periods = []
    in_recession = False
    start_date = None

    for index, row in df.iterrows():
        # Start of a recession
        if row['recession'] and not in_recession:
            start_date = row['date']
            in_recession = True
        # End of a recession
        elif not row['recession'] and in_recession:
            end_date = df.iloc[index - 1]['date']  # previous row's date
            recession_periods.append({'Start': start_date, 'End': end_date})
            in_recession = False
            start_date = None
        # If in recession and it's the last row, end the recession on the last date
        elif in_recession and index == len(df) - 1:
            recession_periods.append({'Start': start_date, 'End': row['date']})

    return recession_periods

# Find the recession periods in the DataFrame
recession_periods = find_recession_periods(df)

# Convert to JSON
recession_json = pd.DataFrame(recession_periods).to_json(orient='records', date_format='iso')

# Output JSON string
recession_json

In [ ]:
# Define the path for the JSON file
json_file_path = '../altair_wrapper/data/recessions_uk.json'

# Save the JSON string to a file
with open(json_file_path, 'w') as file:
    file.write(recession_json)